In [18]:
!pip install langchain-teddynote

In [2]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.15
    Uninstalling langchain-core-0.3.15:
      Successfully uninstalled langchain-core-0.3.15


In [19]:
!pip install langchain_huggingface

In [20]:
import os

from google.colab import userdata

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "LangChain 실습"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('langsmith_api_key')

In [21]:
import zipfile

# 데이터 처리 및 분석
import pandas as pd
import numpy as np
from scipy import stats
from tqdm import tqdm
import seaborn as sns
from matplotlib import pyplot as plt
import datetime

from sklearn.metrics.pairwise import cosine_similarity

# 모델 저장
import joblib

# To ignore all warnings
import warnings
warnings.filterwarnings('ignore')



In [22]:
from langchain_teddynote import logging

logging.langsmith("CH01-Basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


In [23]:
from langchain_community.chat_models import ChatPerplexity
from langchain_core.prompts import ChatPromptTemplate

os.environ["PPLX_API_KEY"] = userdata.get('perplexity_api_key')


In [77]:
chat = ChatPerplexity(temperature=0.1,  model="llama-3.1-sonar-small-128k-chat")

In [78]:
prompt_buykorea = ChatPromptTemplate.from_messages([
    ("system", """Analyze the product description. Identify the product type using HS code-based product categories,
    but express it ONLY as a product category name, NOT as a numerical code.
    Extract 5 relevant keywords focusing on the product's functions and purposes.
    For selecting keywords, make educated guesses based on previously inferred facts and the product category.
    Output as a single sentence in the format: '[HS code-based product type], [keyword1], [keyword2], [keyword3], [keyword4], [keyword5].'
    No adjectives, adverbs, or additional explanations."""),
    ("human", "{input}")
])

chain_buykorea = prompt_buykorea | chat

In [79]:
paragraph = """Medium Buildable Coverage, Natural Matte Finish, Transformative Cream-To-Powder Texture
A tri-system blend of water, emollients and powder glides onto skin as a soft, creamy emulsion with a lightweight, transformative texture.
The innovative sweat- and humidity-resistant formula provides medium buildable coverage, allowing for true-to-skin shading from very light to very deep in a natural matte finish.
Light-diffusing properties soften lines to create a fresh, flawless texture for all skin types.
This formula also delivers immediate and long-term moisturization with 12-hour wear and won't cause acne. Now available in 42 all-inclusive, colour-true shades."""

response_buykorea = chain_buykorea.invoke({"input": paragraph})
print(response_buykorea.content)

Foundation, buildable coverage, natural finish, transformative texture, moisturizing.


In [80]:
prompt_bldata = ChatPromptTemplate.from_messages([
    ("system", """Analyze the product name. Identify the product type using HS code-based product categories,
    but express it ONLY as a product category name, NOT as a numerical code.
    Extract 5 relevant keywords focusing on the product's functions and purposes.
    For selecting keywords, make educated guesses based on previously inferred facts and the product category.
    Output as a single sentence in the format: '[HS code-based product type], [keyword1], [keyword2], [keyword3], [keyword4], [keyword5].'
    No adjectives, adverbs, or additional explanations."""),
    ("human", "{input}")
])

chain_bldata = prompt_bldata | chat

In [86]:
name = """STUDIO TECH-NC25 10GM/.35OZ - (10.00 g Each) - M1K2120000"""

response_bldata = chain_bldata.invoke({"input": name})
print(response_bldata.content)

Cosmetic or Pharmaceutical Product, moisturizing, hydrating, skin care, nourishing, protective.


In [83]:
from langchain_huggingface import HuggingFaceEmbeddings

# HuggingFaceEmbeddings 초기화
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [87]:

# 임베딩할 텍스트
text_buykorea = response_buykorea.content
text_bldata = response_bldata.content

# 텍스트 임베딩
embedding_buykorea = embeddings.embed_query(text_buykorea)
embedding_bldata = embeddings.embed_query(text_bldata)

# 결과 출력
print(f"전처리한 바이코리아 데이터 : {text_buykorea}")
print(f"전처리한 BL 데이터 : {text_bldata}")

print(f"코사인 유사도 : {cosine_similarity(np.array(embedding_buykorea).reshape(1,-1), np.array(embedding_bldata).reshape(1,-1))}")

전처리한 바이코리아 데이터 : Foundation, buildable coverage, natural finish, transformative texture, moisturizing.
전처리한 BL 데이터 : Cosmetic or Pharmaceutical Product, moisturizing, hydrating, skin care, nourishing, protective.
코사인 유사도 : [[0.63029993]]


In [88]:
# 텍스트 임베딩
embedding_buykorea_raw = embeddings.embed_query(paragraph)
embedding_bldata_raw = embeddings.embed_query(name)

# 결과 출력
print(f"원본 바이코리아 데이터 : {paragraph}")
print(f"원본 BL 데이터 : {name}")

print(f"코사인 유사도 : {cosine_similarity(np.array(embedding_buykorea_raw).reshape(1,-1), np.array(embedding_bldata_raw).reshape(1,-1))}")

원본 바이코리아 데이터 : Medium Buildable Coverage, Natural Matte Finish, Transformative Cream-To-Powder Texture
A tri-system blend of water, emollients and powder glides onto skin as a soft, creamy emulsion with a lightweight, transformative texture. 
The innovative sweat- and humidity-resistant formula provides medium buildable coverage, allowing for true-to-skin shading from very light to very deep in a natural matte finish. 
Light-diffusing properties soften lines to create a fresh, flawless texture for all skin types. 
This formula also delivers immediate and long-term moisturization with 12-hour wear and won't cause acne. Now available in 42 all-inclusive, colour-true shades.
원본 BL 데이터 : STUDIO TECH-NC25 10GM/.35OZ - (10.00 g Each) - M1K2120000
코사인 유사도 : [[0.19841966]]
